# Part 0 & 1

# Load dataset

In [1]:
from IPython import embed
from datasets import load_dataset
dataset = load_dataset('rotten_tomatoes')
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

C:\Users\Aozy\anaconda3\envs\sc4002\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = train_dataset.to_pandas()
train_df.head()

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


# Create vocab from training dataset

In [3]:
import nltk


def build_vocab(train_dataset):
    # Create set, unique words only
    vocab = set()
    train_dataset_pos = []
    
    # Loop thru each sentence in training dataset
    for sentence in train_dataset['text']:
        # Basic text processing
        
        # Case folding
        sentence = sentence.lower()
        
        # NLTK tokenizer does a good job at separating meaningful words + punctuations
        # Better than defining regex ourselves
        word_list = nltk.tokenize.word_tokenize(sentence)
        
        # # Further split words into separate words
        # # e.g., 'well-being' -> 'well', 'being'
        # # e.g., 'music/song' -> 'music', 'song'
        # split_word_list = []
        # for word in sentence_list:
        #     split_word_list.extend(word.replace('-', ' ').replace('/', ' ').split())
        
        # Dont remove all special characters, some are meaningful
        # Some words are surrounded by single/double quotes
        word_list = [word.strip("'\"") for word in word_list]
        
        # Add into set
        vocab.update(word_list)
        
        # Get pos tags
        # Also build POS tags
        pos_tags = nltk.pos_tag(word_list)
        train_dataset_pos.append(pos_tags)
        
    vocab.discard('')
    return vocab, train_dataset_pos

In [4]:
vocab, train_dataset_pos = build_vocab(train_dataset)

print(vocab)

{'resoundingly', 'scenes', 'specifics', 'writer-producer-director', 'contend', 'autopilot', 'wildly', 'breaks', 'disjointed', 'released', 'insider', 'dullingly', 'church', 'nostra', 'travails', 'throes', 'brazenly', 'while', 'domestic', 'cornball', 'upends', 'contemplative', 'tautou', 'criminal', 'bigger-than-life', 'feature', 'sleepless', 'enigma', 'interludes', 'white-trash', 'many', 'otherness', 'vivi', 'elegante', 'absurdity', 'saímos', 'ruin', 'agreeable', '12-year-old', 'dated', 'hail', 'extracting', 'emperor', 'innovations', 'culture', 'starters', 'football', 'freudianism', 'angels', 'harshness', 'churlish', 'responsible', 'immaculate', 'uppity', 'tenuous', 'limping', 'alienating', 'cinematographer', 'fire-red', 'sleeper', 'heather', '-after', 'flow', 'preference', 'afternoon', 'dos', 'wacky', 'esos', 'two-actor', 'one', 'lightly', 'shatner', 'implausibility', 'infused', 'prostituted', 'stunning', 'lucks', 'burrito', 'handling', 'funcionar', 'writer/director/producer', 'official

In [5]:
print(train_dataset_pos[0])

[('the', 'DT'), ('rock', 'NN'), ('is', 'VBZ'), ('destined', 'VBN'), ('to', 'TO'), ('be', 'VB'), ('the', 'DT'), ('21st', 'JJ'), ('century', 'NN'), ('s', 'VBD'), ('new', 'JJ'), ('``', '``'), ('conan', 'JJ'), ('``', '``'), ('and', 'CC'), ('that', 'IN'), ('he', 'PRP'), ('s', 'VBZ'), ('going', 'VBG'), ('to', 'TO'), ('make', 'VB'), ('a', 'DT'), ('splash', 'NN'), ('even', 'RB'), ('greater', 'JJR'), ('than', 'IN'), ('arnold', 'RB'), ('schwarzenegger', 'NN'), (',', ','), ('jean-claud', 'JJ'), ('van', 'NN'), ('damme', 'NN'), ('or', 'CC'), ('steven', 'JJ'), ('segal', 'NN'), ('.', '.')]


## (a) What is the size of the vocabulary formed from your training data?

In [6]:
print(f'Vocab size/Unique words: {len(vocab)}')

Vocab size/Unique words: 17841


# Part 1: Preparing Word Embeddings

## Download GloVe embeddings: https://nlp.stanford.edu/projects/glove/
- Uncased means all words are lowercase

In [7]:
# Load GloVe embeddings
import numpy as np

def load_glove_embeddings(path):
    glove_embeddings = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float64')
            glove_embeddings[word] = vector
            
    return glove_embeddings

In [8]:
glove_embeddings = load_glove_embeddings('glove.6B.50d.txt')

In [9]:
print(f'Glove embedding matrix for "the":\n {glove_embeddings["the"]}')

Glove embedding matrix for "the":
 [ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]


In [10]:
# Print size of the matrix
print(f'Number of unique words in embedding matrix: {len(glove_embeddings)}')

print(f'Number of Dimension/Features of Glove embedding matrix: {glove_embeddings["the"].shape[0]}')

Number of unique words in embedding matrix: 400000
Number of Dimension/Features of Glove embedding matrix: 50


## Create embedding matrix

In [11]:
# Finalize vocab
vocab_word_to_index = {word: idx for idx, word in enumerate(vocab)}

In [12]:
vocab_word_to_index

{'resoundingly': 0,
 'scenes': 1,
 'specifics': 2,
 'writer-producer-director': 3,
 'contend': 4,
 'autopilot': 5,
 'wildly': 6,
 'breaks': 7,
 'disjointed': 8,
 'released': 9,
 'insider': 10,
 'dullingly': 11,
 'church': 12,
 'nostra': 13,
 'travails': 14,
 'throes': 15,
 'brazenly': 16,
 'while': 17,
 'domestic': 18,
 'cornball': 19,
 'upends': 20,
 'contemplative': 21,
 'tautou': 22,
 'criminal': 23,
 'bigger-than-life': 24,
 'feature': 25,
 'sleepless': 26,
 'enigma': 27,
 'interludes': 28,
 'white-trash': 29,
 'many': 30,
 'otherness': 31,
 'vivi': 32,
 'elegante': 33,
 'absurdity': 34,
 'saímos': 35,
 'ruin': 36,
 'agreeable': 37,
 '12-year-old': 38,
 'dated': 39,
 'hail': 40,
 'extracting': 41,
 'emperor': 42,
 'innovations': 43,
 'culture': 44,
 'starters': 45,
 'football': 46,
 'freudianism': 47,
 'angels': 48,
 'harshness': 49,
 'churlish': 50,
 'responsible': 51,
 'immaculate': 52,
 'uppity': 53,
 'tenuous': 54,
 'limping': 55,
 'alienating': 56,
 'cinematographer': 57,
 'fi

In [13]:
def create_embedding_matrix(word_to_index, glove_embeddings):
    # Initialize embedding matrix with zeros
    # 50d
    embedding_matrix = np.zeros((len(vocab), 50), dtype='float64')
    
    # Loop thru each word in vocab
    for word, idx in word_to_index.items():
        # Check if word exists in glove embeddings
        if word in glove_embeddings:
            # Copy glove embedding to embedding matrix
            embedding_matrix[idx] = glove_embeddings[word]
            # If OOV, assign None first
            
    return embedding_matrix

In [14]:
embedding_matrix = create_embedding_matrix(vocab_word_to_index, glove_embeddings)

In [15]:
embedding_matrix

array([[-0.082129, -1.7836  , -0.048594, ...,  0.70148 ,  0.5154  ,
        -0.274   ],
       [ 0.89474 ,  0.26706 , -0.99777 , ..., -0.66815 , -0.2774  ,
        -0.23598 ],
       [ 1.0701  ,  0.20411 , -0.30751 , ...,  2.3549  , -0.085246,
         0.542   ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [-0.57908 ,  0.52627 ,  0.52603 , ..., -0.37458 ,  0.24555 ,
         0.72296 ],
       [-0.18194 , -0.86901 , -0.51548 , ...,  0.477   ,  0.027697,
         0.42138 ]])

## (b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [16]:
def get_oov_words(embedding_matrix, vocab_word_to_index):
    oov_words = []
    
    for word, idx in vocab_word_to_index.items():
        # Check if zero matrix
        if np.sum(embedding_matrix[idx]) == 0:
            oov_words.append(word)

    return oov_words

In [17]:
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

In [18]:
oov_words.sort()
oov_words[:200]

['-a',
 '-after',
 '-doing-it-for',
 '-greaseballs',
 '-hollywood',
 '-inevitable',
 '-of-the-week',
 '-stunning',
 '-the-cash',
 '-the-night',
 '-west',
 '-white',
 '1/2-hour',
 '102-minute',
 '10th-grade',
 '112-minute',
 '129-minute',
 '170-minute',
 '179-minute',
 '28k',
 '3-year-olds',
 '3/4th',
 '4/5ths',
 '60s-homage',
 '79-minute',
 '8217',
 '99-minute',
 'a-bornin',
 'a-knocking',
 'abandone',
 'aborbing',
 'absolutamente',
 'aburrido',
 'acabamos',
 'accomodates',
 'accordion/harmonica/banjo',
 'aceitou',
 'achival',
 'achronological',
 'acontecimentos',
 'acting-workshop',
 'action-fantasy',
 'action-filled',
 'action-movie',
 'action-thriller/dark',
 'action/comedy',
 'action/effects',
 'actorish',
 'actory',
 'actress-producer',
 'actuación',
 'actuada',
 'adapted-',
 'addessi',
 'adventues',
 'affectation-free',
 'affirmational',
 'again-courage',
 'age-inspired',
 'age-wise',
 'ages-old',
 "ain't-",
 'alientation',
 'all-in-all',
 'all-over-the-map',
 'all-too',
 'all-to

In [19]:
print(f"Number of OOV words: {len(oov_words)}")

Number of OOV words: 1668


In [20]:
train_dataset_pos

[[('the', 'DT'),
  ('rock', 'NN'),
  ('is', 'VBZ'),
  ('destined', 'VBN'),
  ('to', 'TO'),
  ('be', 'VB'),
  ('the', 'DT'),
  ('21st', 'JJ'),
  ('century', 'NN'),
  ('s', 'VBD'),
  ('new', 'JJ'),
  ('``', '``'),
  ('conan', 'JJ'),
  ('``', '``'),
  ('and', 'CC'),
  ('that', 'IN'),
  ('he', 'PRP'),
  ('s', 'VBZ'),
  ('going', 'VBG'),
  ('to', 'TO'),
  ('make', 'VB'),
  ('a', 'DT'),
  ('splash', 'NN'),
  ('even', 'RB'),
  ('greater', 'JJR'),
  ('than', 'IN'),
  ('arnold', 'RB'),
  ('schwarzenegger', 'NN'),
  (',', ','),
  ('jean-claud', 'JJ'),
  ('van', 'NN'),
  ('damme', 'NN'),
  ('or', 'CC'),
  ('steven', 'JJ'),
  ('segal', 'NN'),
  ('.', '.')],
 [('the', 'DT'),
  ('gorgeously', 'RB'),
  ('elaborate', 'JJ'),
  ('continuation', 'NN'),
  ('of', 'IN'),
  ('``', '``'),
  ('the', 'DT'),
  ('lord', 'NN'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('rings', 'NNS'),
  ('``', '``'),
  ('trilogy', 'NN'),
  ('is', 'VBZ'),
  ('so', 'RB'),
  ('huge', 'JJ'),
  ('that', 'IN'),
  ('a', 'DT'),
  ('column', 'N

In [21]:
# Find POS tags for OOV words
oov_words_pos = {}
for oov_word in oov_words:
    # Find all possible POS for OOV word
    pos_tags = []
    for sentence_pos in train_dataset_pos:
        for word, pos in sentence_pos:
            if word == oov_word:
                pos_tags.append(pos)
    # # Assign
    oov_words_pos[oov_word] = pos_tags

In [22]:
oov_words_pos

{'-a': ['NNP'],
 '-after': ['NNP'],
 '-doing-it-for': ['JJ'],
 '-greaseballs': ['NNS'],
 '-hollywood': ['NN'],
 '-inevitable': ['JJ'],
 '-of-the-week': ['JJ'],
 '-stunning': ['NN'],
 '-the-cash': ['NNP'],
 '-the-night': ['NN'],
 '-west': ['JJS'],
 '-white': ['NN'],
 '1/2-hour': ['JJ'],
 '102-minute': ['JJ'],
 '10th-grade': ['JJ'],
 '112-minute': ['JJ'],
 '129-minute': ['JJ'],
 '170-minute': ['JJ'],
 '179-minute': ['JJ'],
 '28k': ['CD'],
 '3-year-olds': ['NNS'],
 '3/4th': ['CD'],
 '4/5ths': ['CD'],
 '60s-homage': ['JJ'],
 '79-minute': ['JJ'],
 '8217': ['CD', 'CD'],
 '99-minute': ['JJ'],
 'a-bornin': ['JJ'],
 'a-knocking': ['NN'],
 'abandone': ['NN'],
 'aborbing': ['VBG'],
 'absolutamente': ['NN'],
 'aburrido': ['NN'],
 'acabamos': ['NN'],
 'accomodates': ['VBZ'],
 'accordion/harmonica/banjo': ['NN'],
 'aceitou': ['NN'],
 'achival': ['JJ'],
 'achronological': ['NN', 'JJ'],
 'acontecimentos': ['NNS'],
 'acting-workshop': ['JJ'],
 'action-fantasy': ['JJ'],
 'action-filled': ['JJ'],
 'actio

## (c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

## Pass 1: Use Stemming to match OOV words of different forms to the same word in GloVe

In [23]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

# Stem vocab
stemmed_glove_vocab = {ps.stem(word): word for word in glove_embeddings.keys()}

# Stem vocab and OOV words, find same word
def find_substitute_word_stem(oov_word, stemmed_glove_vocab):
    stemmed_oov_word = ps.stem(oov_word)
    if stemmed_oov_word in stemmed_glove_vocab.keys():
        return stemmed_glove_vocab[stemmed_oov_word]
    
    return None

In [24]:
# Copy substitute word embedding to OOV word
for oov_word in oov_words:
    best_substitute_word = find_substitute_word_stem(oov_word, stemmed_glove_vocab)
    if best_substitute_word:
        print(f'OOV word: {oov_word}, substitute word: {best_substitute_word}')
        embedding_matrix[vocab_word_to_index[oov_word]] = glove_embeddings[best_substitute_word]

OOV word: 3-year-olds, substitute word: 3-year-old
OOV word: abandone, substitute word: abandonned
OOV word: accomodates, substitute word: accomodative
OOV word: aceitou, substitute word: aceite
OOV word: affirmational, substitute word: affirmance
OOV word: allodi, substitute word: allodial
OOV word: anteing, substitute word: anting
OOV word: anti-date, substitute word: anti-de
OOV word: aqueles, substitute word: aquel
OOV word: birot, substitute word: birote
OOV word: bjorkness, substitute word: bjorke
OOV word: bug-eye, substitute word: bug-eyed
OOV word: butterfingered, substitute word: butterfingers
OOV word: by-the-books, substitute word: by-the-book
OOV word: cadness, substitute word: cads
OOV word: capturou, substitute word: captures
OOV word: cat-and-mouser, substitute word: cat-and-mouse
OOV word: cineasts, substitute word: cineaste
OOV word: co-dependence, substitute word: co-dependent
OOV word: colonics, substitute word: colonizations
OOV word: complejos, substitute word: co

In [25]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 1527


## Pass 2: Use Wordnet synonyms bank based on sentence POS tagging

In [26]:
oov_words_pos

{'-a': ['NNP'],
 '-after': ['NNP'],
 '-doing-it-for': ['JJ'],
 '-greaseballs': ['NNS'],
 '-hollywood': ['NN'],
 '-inevitable': ['JJ'],
 '-of-the-week': ['JJ'],
 '-stunning': ['NN'],
 '-the-cash': ['NNP'],
 '-the-night': ['NN'],
 '-west': ['JJS'],
 '-white': ['NN'],
 '1/2-hour': ['JJ'],
 '102-minute': ['JJ'],
 '10th-grade': ['JJ'],
 '112-minute': ['JJ'],
 '129-minute': ['JJ'],
 '170-minute': ['JJ'],
 '179-minute': ['JJ'],
 '28k': ['CD'],
 '3-year-olds': ['NNS'],
 '3/4th': ['CD'],
 '4/5ths': ['CD'],
 '60s-homage': ['JJ'],
 '79-minute': ['JJ'],
 '8217': ['CD', 'CD'],
 '99-minute': ['JJ'],
 'a-bornin': ['JJ'],
 'a-knocking': ['NN'],
 'abandone': ['NN'],
 'aborbing': ['VBG'],
 'absolutamente': ['NN'],
 'aburrido': ['NN'],
 'acabamos': ['NN'],
 'accomodates': ['VBZ'],
 'accordion/harmonica/banjo': ['NN'],
 'aceitou': ['NN'],
 'achival': ['JJ'],
 'achronological': ['NN', 'JJ'],
 'acontecimentos': ['NNS'],
 'acting-workshop': ['JJ'],
 'action-fantasy': ['JJ'],
 'action-filled': ['JJ'],
 'actio

In [27]:
from nltk.corpus import wordnet as wn

pos_mapping_dict = {'NN':wn.NOUN,
              'JJ':wn.ADJ,
              'VB':wn.VERB,
              'RB':wn.ADV,
              # NLTK does not have wn.ADV_SAT
              }

# Convert oov_words_pos to wordnet POS
def map_pos_list(pos_list, mapping_dict):
    mapped_pos = []
    for pos in pos_list:
        # Find the first matching POS tag in mapping_dict by first letter comparison
        matched_pos = next((mapping_dict[key] for key in mapping_dict if key[0] == pos[0]), None)
        mapped_pos.append(matched_pos)
    return mapped_pos

# Creating the pos list
oov_words_pos_wordnet = {}
for key, pos_list in oov_words_pos.items():
    mapped_pos_list = map_pos_list(pos_list, pos_mapping_dict)
    
    # Remove None from list
    cleaned_list = [pos for pos in mapped_pos_list if pos is not None]
    
    # If all nones, remove whole entry
    if cleaned_list:
        oov_words_pos_wordnet[key] = cleaned_list
    

# Print the new dictionary
for key, value in oov_words_pos_wordnet.items():
    print(f"{key}: {value}")

-a: ['n']
-after: ['n']
-doing-it-for: ['a']
-greaseballs: ['n']
-hollywood: ['n']
-inevitable: ['a']
-of-the-week: ['a']
-stunning: ['n']
-the-cash: ['n']
-the-night: ['n']
-west: ['a']
-white: ['n']
1/2-hour: ['a']
102-minute: ['a']
10th-grade: ['a']
112-minute: ['a']
129-minute: ['a']
170-minute: ['a']
179-minute: ['a']
3-year-olds: ['n']
60s-homage: ['a']
79-minute: ['a']
99-minute: ['a']
a-bornin: ['a']
a-knocking: ['n']
abandone: ['n']
aborbing: ['v']
absolutamente: ['n']
aburrido: ['n']
acabamos: ['n']
accomodates: ['v']
accordion/harmonica/banjo: ['n']
aceitou: ['n']
achival: ['a']
achronological: ['n', 'a']
acontecimentos: ['n']
acting-workshop: ['a']
action-fantasy: ['a']
action-filled: ['a']
action-movie: ['a', 'a']
action-thriller/dark: ['a']
action/comedy: ['n']
action/effects: ['n']
actorish: ['a']
actory: ['a']
actress-producer: ['n']
actuada: ['n', 'a']
adapted-: ['a']
addessi: ['n']
adventues: ['n']
affectation-free: ['a']
affirmational: ['a']
again-courage: ['a']
age-

In [28]:
def find_substitute_wordnet_synonym(oov_word, oov_words_pos_wordnet):
    # Find synonyms for OOV words
    # Retrieve POS tags list
    try:
        pos_list = oov_words_pos_wordnet[oov_word]
    except KeyError:
        return None
    
    # Find number of unique pos, except None
    unique_pos = list(set(pos_list))
    # sort by count
    unique_pos.sort(key=lambda x: pos_list.count(x), reverse=True)
    
    # Loop thru each unique pos, try to find synonyms
    for possible_pos_tag in unique_pos:
        # Get synonyms
        for synset in wn.synsets(oov_word, pos=possible_pos_tag):
            for lemma in synset.lemmas():
                if lemma.name() in glove_embeddings:
                    print(f'OOV word: {oov_word}, synonym: {lemma.name()}')
                    return glove_embeddings[lemma.name()]

In [29]:
for oov_word in oov_words:
    synonym_glove_embedding = find_substitute_wordnet_synonym(oov_word, oov_words_pos_wordnet)
    if synonym_glove_embedding is not None:
        embedding_matrix[vocab_word_to_index[oov_word]] = synonym_glove_embedding

OOV word: creepy-crawly, synonym: creepy
OOV word: hastier, synonym: hasty
OOV word: punch-drunk, synonym: silly
OOV word: hoo-ha, synonym: disturbance
OOV word: off-the-rack, synonym: off-the-shelf
OOV word: rip-roaring, synonym: uproarious
OOV word: juiceless, synonym: dry
OOV word: greasiest, synonym: greasy
OOV word: dewy-eyed, synonym: childlike
OOV word: corniest, synonym: corny
OOV word: hopped-up, synonym: stoned
OOV word: sappier, synonym: cockamamie
OOV word: gabbiest, synonym: chatty
OOV word: ho-hum, synonym: boring
OOV word: perfervid, synonym: ardent
OOV word: captivatingly, synonym: enchantingly
OOV word: cannier, synonym: cagey
OOV word: excrescence, synonym: bulge
OOV word: hoity-toity, synonym: grandiose
OOV word: fine-looking, synonym: good-looking
OOV word: self-involved, synonym: self-absorbed
OOV word: ham-fisted, synonym: bumbling
OOV word: fuddled, synonym: befuddle
OOV word: shmear, synonym: schmear
OOV word: divertingly, synonym: amusingly
OOV word: ankle-deep

In [30]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 1489


## Pass 3: Use Edit Distance to solve misspelled OOV words

In [31]:
from Levenshtein import distance as lev

# Find most similar word for OOV word
def find_substitute_word_edit_dist(oov_word, glove_embeddings):
    # Set to infinity first
    min_dist = float('inf')
    closest_word = None
    
    # Loop thru all words in glove embeddings
    for word in glove_embeddings:
        # Calculate edit distance
        dist = lev(oov_word, word)
        
        # Update if new minimum distance found
        if dist < min_dist:
            min_dist = dist
            closest_word = word
            
    return closest_word, min_dist

In [32]:
min_dist_thresh = 1

for oov_word in oov_words:
    best_substitute_word, min_dist = find_substitute_word_edit_dist(oov_word, glove_embeddings)
    if min_dist <= min_dist_thresh:
        print(f'OOV word: {oov_word}, substitute word: {best_substitute_word}, Distance: {min_dist}')
        # Copy substitute word embedding to OOV word
        embedding_matrix[vocab_word_to_index[oov_word]] = glove_embeddings[best_substitute_word]

OOV word: saímos, substitute word: samos, Distance: 1
OOV word: fire-red, substitute word: firered, Distance: 1
OOV word: -after, substitute word: after, Distance: 1
OOV word: two-actor, substitute word: two-factor, Distance: 1
OOV word: funcionar, substitute word: funciona, Distance: 1
OOV word: 1/2-hour, substitute word: 12-hour, Distance: 1
OOV word: dridi, substitute word: didi, Distance: 1
OOV word: memória, substitute word: memoria, Distance: 1
OOV word: têm, substitute word: tom, Distance: 1
OOV word: mid-section, substitute word: midsection, Distance: 1
OOV word: prep-school, substitute word: pre-school, Distance: 1
OOV word: papai, substitute word: papal, Distance: 1
OOV word: show-tunes, substitute word: showtunes, Distance: 1
OOV word: 112-minute, substitute word: 12-minute, Distance: 1
OOV word: character-, substitute word: character, Distance: 1
OOV word: pianista, substitute word: pianist, Distance: 1
OOV word: clutchy, substitute word: clutch, Distance: 1
OOV word: volle

# Remaining OOV words

In [33]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 1161


## Last pass: Replace OOV words with <UNK> token

In [38]:
unk_token = '<UNK>'
# Random embedding for <UNK> token
embedding_dim = 50
unk_embedding = np.random.uniform(-0.25, 0.25, embedding_dim)

# Assign <UNK> token embedding to OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)
for oov_word in oov_words:
    embedding_matrix[vocab_word_to_index[oov_word]] = unk_embedding
    
# Add <UNK> token to vocab and embedding matrix
vocab_word_to_index[unk_token] = len(vocab)
embedding_matrix = np.vstack([embedding_matrix, unk_embedding])

In [41]:
embedding_matrix[17841]

array([ 1.80637154e-01, -1.00917729e-01,  6.92024745e-02, -1.02085042e-01,
       -2.04966966e-01, -1.42269065e-01, -1.75927993e-01,  2.15488091e-01,
        1.78611817e-01,  7.53884473e-02,  2.98657557e-02, -6.45055946e-02,
       -1.66512879e-01, -1.77123642e-01,  1.82497188e-02, -5.04283763e-02,
        1.95425265e-02,  1.15639105e-01, -5.69365739e-02, -1.18176631e-01,
       -1.87504417e-01, -1.52666837e-01, -1.18570521e-01, -9.82018151e-02,
       -1.26744825e-01,  1.82820383e-01,  1.27015259e-01, -9.01787711e-02,
        8.81058353e-02,  1.29658365e-01, -1.33080964e-01,  2.34593900e-01,
        6.62886136e-05,  1.26643207e-01,  1.83764196e-01,  7.15199850e-02,
       -1.18927113e-01, -2.33772635e-01,  2.04115945e-01, -3.88743676e-02,
        1.25969900e-01,  2.42054301e-01,  1.58390327e-01, -2.10504457e-01,
        1.12277197e-01,  1.70272325e-01,  1.14982568e-01,  1.39461121e-01,
       -2.02386915e-02,  2.06038250e-01])

## Alternative methods to test
- FastText, break words into subwords and use subword embeddings
- semantic similarity with cosine similarity
- use model to predict oov embeddings Mimick https://github.com/yuvalpinter/Mimick


# Finally, save embedding matrix and vocab_to_index mapping

In [42]:
import pickle


with open('embedding_matrix.pkl', 'wb') as f:
    pickle.dump(embedding_matrix, f)
    
with open('vocab_word_to_index.pkl', 'wb') as f:
    pickle.dump(vocab_word_to_index, f)